### This was an attempt at putting together a smaller subset of data for just MN. I did find college attendence rates that I had hoped to join with the national school data. The problem of matching on a string again arose. 

In [6]:
%matplotlib inline

from __future__ import print_function
from statsmodels.compat import lzip
import statsmodels
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
from scipy import stats
import pickle

from statsmodels.stats.outliers_influence import OLSInfluence

### Steps:
#### 1. Subset out features of interest
#### 2. Clean unprocessable values ('-', 'N/A', etc)
#### 3. Format column data types
#### 4. Create new features
#### 5. Fill empty values with NANs
#### 6. Drop outliers
#### 7. Join datasets
#### 8. Subset out features of interst (2)
#### 9. Transform*
#### 10. Create dummy variables*

In [4]:
mn_import_df = pd.read_csv('../data/interim/2014-2015 Graduates.csv')
print(mn_import_df.shape)
mn_import_df.head()

(1212, 24)


,dat_yer,dst_num,dst_tye,dst_nam,sch_num,sch_nam,ami_mal,ami_fem,api_mal,api_fem,...,wht_mal,wht_fem,tot_mal,tot_fem,tot_mny,tot_tot,cou_num,cou_nam,sch_cls,eco_dev
0,14-15,1.0,1.0,AITKIN PUBLIC SCHOOL DISTRICT,1.0,AITKIN SECONDARY SCHOOL,0.0,0.0,0.0,0.0,...,36.0,43.0,37.0,44.0,2.0,81.0,01,Aitkin,33,03
1,14-15,1.0,1.0,AITKIN PUBLIC SCHOOL DISTRICT,6.0,AITKIN ALTERNATIVE LEARNING PROGRAM,0.0,0.0,0.0,0.0,...,1.0,3.0,1.0,3.0,0.0,4.0,01,Aitkin,42,03
2,14-15,1.0,1.0,AITKIN PUBLIC SCHOOL DISTRICT,0.0,District Totals,0.0,0.0,0.0,0.0,...,37.0,46.0,38.0,47.0,2.0,85.0,01,Aitkin,,03
3,14-15,1.0,3.0,MINNEAPOLIS PUBLIC SCHOOL DIST.,307.0,BASIC SKILLS-4,0.0,1.0,3.0,3.0,...,16.0,9.0,46.0,39.0,60.0,85.0,27,Hennepin,41,11
4,14-15,1.0,3.0,MINNEAPOLIS PUBLIC SCHOOL DIST.,348.0,MERC,0.0,0.0,0.0,0.0,...,1.0,0.0,13.0,9.0,21.0,22.0,27,Hennepin,43,11


In [49]:
with open('../data/processed/0216_elsi_public_schools.pkl', 'rb') as picklefile: 
    elsi_df = pickle.load(picklefile)
print(elsi_df.shape)
elsi_df.head()

(18114, 59)


,school_name,state_name,state,school_name_cl,school_id,agency_id,county_name,county_number,fips_state_code,school_level_code,...,hawaiian_nat_pacific_isl_students,two_or_more_races_students,total_race_ethnicity,male_students,female_students,agency_type_dis_x,school_type_x,state_school_id_x,congressional_code_x,reconstituted_flag
1,1050 ADAIR CO. HIGH,Missouri,MO,ADAIR CO. HIGH,290579000125,2905790,ADAIR COUNTY,29001,29,3-High,...,"=""0""",2,†,41,51,1-Regular local school district that is NOT a ...,1-Regular school,1050001092,2906,2-No
2,112 ALC AFTER SCHOOL & SUMMER SCH,Minnesota,MN,112 ALC AFTER SCHOOL & SUMMER SCH,270819004415,2708190,CARVER COUNTY,27019,27,3-High,...,"=""0""",1,†,10,2,1-Regular local school district that is NOT a ...,4-Alternative/other school,"=""010112067""",2703,2-No
7,21ST CENTURY ACADEMY,Kentucky,KY,21st Century Academy,210115002241,2101150,CHRISTIAN COUNTY,21047,21,3-High,...,"=""0""",3,†,52,41,1-Regular local school district that is NOT a ...,4-Alternative/other school,"=""024115185""",2101,2-No
14,21ST CENTURY SKILLS ACADEMY,Massachusetts,MA,21st Century Skills Academy,251251002790,2512510,HAMPDEN COUNTY,25013,25,3-High,...,"=""0""","=""0""",†,9,4,1-Regular local school district that is NOT a ...,1-Regular school,"=""03320515""",2501,2-No
15,270 - HAP - IS,Minnesota,MN,270 - HAP - IS,270002504573,2700025,HENNEPIN COUNTY,27053,27,3-High,...,"=""0""","=""0""",†,"=""0""","=""0""",4-Regional Education Service Agency (RESA),4-Alternative/other school,"=""060287794""",2705,2-No


In [50]:
elsi_df = elsi_df[elsi_df['state'] == 'MN']

#### 1. Subset out features of interest

In [13]:
# Get only needed cols from dataset
mn_cols = ['dst_nam', 'sch_nam', 'tot_tot']

In [18]:
mn_subset1_df = mn_import_df[mn_cols].copy(deep=True)

In [20]:
mn_subset1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 3 columns):
dst_nam    1211 non-null object
sch_nam    1211 non-null object
tot_tot    1211 non-null float64
dtypes: float64(1), object(2)
memory usage: 28.5+ KB


In [51]:
# Get only needed cols from dataset
elsi_cols = ['school_name',
       'state',\
       'county_name',\
       'school_level_code',\
       'charter_school',\
       'magnet_school',\
       'shared_time_school',\
       'urban_centric_locale',\
       'school_type',\
       'free_lunch_eligible',\
       'reduced_price_lunch_eligible_students',\
       'pupil_teacher_ratio',\
       'address',\
       'city',\
       'zip',\
       'american_indian_students',\
       'asian_or_asian_pacif_isl_students',\
       'hispanic_students',\
       'black_students',\
       'white_students',\
       'hawaiian_nat_pacific_isl_students',\
       'two_or_more_races_students',\
       'total_race_ethnicity',\
       'male_students',\
       'female_students']

In [53]:
elsi_subset1_df = elsi_df[elsi_cols]

In [54]:
elsi_subset1_df = elsi_subset1_df.copy(deep=True)

#### 2.Join datasets

In [57]:
# Join datasets
import_df = pd.merge(mn_subset1_df, mn_elsi_subset1, how='inner', left_on='sch_nam', right_on='school_name')
import_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 28 columns):
dst_nam                                  11 non-null object
sch_nam                                  11 non-null object
tot_tot                                  11 non-null float64
school_name                              11 non-null object
state                                    11 non-null object
county_name                              11 non-null object
school_level_code                        11 non-null object
charter_school                           11 non-null object
magnet_school                            11 non-null object
shared_time_school                       11 non-null object
urban_centric_locale                     11 non-null object
school_type                              11 non-null object
free_lunch_eligible                      11 non-null object
reduced_price_lunch_eligible_students    11 non-null object
pupil_teacher_ratio                      11 non-null obj

<em> 11 matches with the join. This isn't worth pursuing further. </em>

#### 2. Clean unprocessable values ('-', 'N/A', etc)

In [ ]:
subset1_df.replace('','', inplace=True, regex=True)

#### 3. Format column data types

In [ ]:
float_cols = []

int_cols = []

In [ ]:
# Convert rows to float

cast_as_float(join_id_df, float_cols)

In [ ]:
# Convert rows to int

cast_as_int(subset1_df_df, int_cols)

#### 4. Create new features

#### 5. Fill empty values with NANs

In [ ]:
# Fill NA values

fillna_with_mean(subset1_df, [cols])
fillna_with_median(subset1_df, [cols])

#### 6. Drop outliers and duplicates

In [ ]:
elsi_df.drop_duplicates(subset=[], keep='first', inplace=True)

#### 8. Subset out features of inter3st (2)

In [ ]:
subset2_df = raw_elsi_df[raw_elsi_df['school_level_code'] == '3-High']
elsi_df.info()

#### 9. Transform*

#### 10. Create dummy variables*

In [ ]:
dummies = []

In [ ]:
wdum_df = pd.get_dummies(df, columns=dummies, drop_first=True)